<a href="https://colab.research.google.com/github/bbberylll/ESAA_OB/blob/main/ESAA_OB2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')

train_final = pd.read_csv('/content/drive/MyDrive/train.csv')
test_final = pd.read_csv('/content/drive/MyDrive/test.csv')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re, html
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from nltk import pos_tag
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [ ]:
def nltk_pos_to_wn(pos):
    if pos.startswith('J'): return wordnet.ADJ
    if pos.startswith('V'): return wordnet.VERB
    if pos.startswith('N'): return wordnet.NOUN
    if pos.startswith('R'): return wordnet.ADV
    return wordnet.NOUN

def clean_and_lemma_pos(text: str) -> str:
    text = html.unescape(str(text))
    text = re.sub(r"http[s]?://\S+|www\.\S+", " ", text)
    text = re.sub(r"@\w+", " ", text)
    text = re.sub(r"#(\w+)", r"\1", text)
    text = text.lower()
    text = re.sub(r"[^a-z\s]", " ", text)
    text = re.sub(r"\s{2,}", " ", text).strip()
    toks = text.split()
    tagged = pos_tag(toks)
    return " ".join(lemmatizer.lemmatize(w, nltk_pos_to_wn(p)) for w,p in tagged)

train_final['clean_text_lemma2'] = train_final['text'].apply(clean_and_lemma_pos)
test_final['clean_text_lemma2'] = test_final['text'].apply(clean_and_lemma_pos)

In [ ]:
def normalize_repeats(text):
    # 문자 반복을 2번으로 제한: e.g., cooool → cool
    return re.sub(r'(.)\1{2,}', r'\1\1', text)

train_final['clean_text_final'] = train_final['clean_text_lemma2'].apply(normalize_repeats)
test_final['clean_text_final'] = test_final['clean_text_lemma2'].apply(normalize_repeats)

In [ ]:
print(train_final.shape)
test_final.shape

(7613, 7)


(3263, 6)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text as sktext

domain_stop = {
    'aba',
    'abbswinston'
}
## aba, abbswinston이 사라지지 않아서 custom 버전으로 수정함

custom_stop = list(sktext.ENGLISH_STOP_WORDS.union(domain_stop))


count_vec_f = CountVectorizer(
    stop_words=custom_stop,
    ngram_range=(1,2),
    min_df=5,
    max_df=0.9,
    token_pattern=r'(?u)\b[a-z]{3,}\b'
)



In [ ]:
train_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 7613 non-null   int64 
 1   keyword            7552 non-null   object
 2   location           5080 non-null   object
 3   text               7613 non-null   object
 4   target             7613 non-null   int64 
 5   clean_text_lemma2  7613 non-null   object
 6   clean_text_final   7613 non-null   object
dtypes: int64(2), object(5)
memory usage: 416.5+ KB


In [ ]:
## 데이터셋 분리
from sklearn.model_selection import train_test_split

X = train_final['clean_text_final']
y = train_final['target']

X_train, X_val, y_train, y_val = train_test_split(X, y,
                                                    test_size=0.2, random_state=42, stratify=y)

In [ ]:
X_count = count_vec_f.fit_transform(train_final['clean_text_final'])

X_tr, X_v, y_tr, y_v = train_test_split(X_count, y, test_size=0.2, random_state=42, stratify=y)

## 다항 나이브 베이즈

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import f1_score, classification_report

pipe = make_pipeline(count_vec_f, MultinomialNB())

pipe.fit(X_train, y_train)
MNB_pred_val = pipe.predict(X_val)
MNB_f1 = f1_score(y_val, MNB_pred_val, average="macro")
print(f"MNB F1 (val) : {MNB_f1:.4f}")

MNB F1 (val) : 0.8003


In [ ]:
print(classification_report(y_val, MNB_pred_val))

              precision    recall  f1-score   support

           0       0.80      0.89      0.84       869
           1       0.83      0.70      0.76       654

    accuracy                           0.81      1523
   macro avg       0.81      0.80      0.80      1523
weighted avg       0.81      0.81      0.81      1523



In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'countvectorizer__ngram_range' : [(1,1), (1,2)],
    #'countvectorizer__min_df' : [2,5],
    #'countvectorizer__max_df' : [0.85, 0.9],
    #'countvectorizer__max_features' : [None, 30000, 50000],
    'multinomialnb__alpha' : [0.03, 0.1, 0.3, 0.7, 1.0, 1.5, 2.0, 3.0]
    }

grid = GridSearchCV(pipe, param_grid, cv=5, scoring='f1_macro', n_jobs=-1)

grid.fit(X_train, y_train)

print("Best params : ", grid.best_params_)
print("Best f1 : ", grid.best_score_)

Best params :  {'countvectorizer__ngram_range': (1, 2), 'multinomialnb__alpha': 1.0}
Best f1 :  0.7782812134441903


In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'countvectorizer__ngram_range' : [(1,1), (1,2)],
    'countvectorizer__min_df' : [2,5],
    #'countvectorizer__max_df' : [0.85, 0.9],
    #'countvectorizer__max_features' : [None, 30000, 50000],
    'multinomialnb__alpha' : [0.03, 0.1, 0.3, 0.7, 1.0, 1.5, 2.0, 3.0]
    }

grid = GridSearchCV(pipe, param_grid, cv=5, scoring='f1_macro', n_jobs=-1)

grid.fit(X_train, y_train)

print("Best params : ", grid.best_params_)
print("Best f1 : ", grid.best_score_)

Best params :  {'countvectorizer__min_df': 2, 'countvectorizer__ngram_range': (1, 2), 'multinomialnb__alpha': 2.0}
Best f1 :  0.7847455928601439


In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'countvectorizer__ngram_range' : [(1,1), (1,2)],
    'countvectorizer__min_df' : [2,5],
    'countvectorizer__max_df' : [0.85, 0.9],
    #'countvectorizer__max_features' : [None, 30000, 50000],
    'multinomialnb__alpha' : [0.03, 0.1, 0.3, 0.7, 1.0, 1.5, 2.0, 3.0]
    }

grid = GridSearchCV(pipe, param_grid, cv=5, scoring='f1_macro', n_jobs=-1)

grid.fit(X_train, y_train)

print("Best params : ", grid.best_params_)
print("Best f1 : ", grid.best_score_)

Best params :  {'countvectorizer__max_df': 0.85, 'countvectorizer__min_df': 2, 'countvectorizer__ngram_range': (1, 2), 'multinomialnb__alpha': 2.0}
Best f1 :  0.7847455928601439


In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'countvectorizer__ngram_range' : [(1,1), (1,2)],
    'countvectorizer__min_df' : [2,5],
    'countvectorizer__max_df' : [0.85, 0.9],
    'countvectorizer__max_features' : [None, 30000, 50000],
    'multinomialnb__alpha' : [0.03, 0.1, 0.3, 0.7, 1.0, 1.5, 2.0, 3.0]
    }

grid = GridSearchCV(pipe, param_grid, cv=5, scoring='f1_macro', n_jobs=-1)

grid.fit(X_train, y_train)

print("Best params : ", grid.best_params_)
print("Best f1 : ", grid.best_score_)

Best params :  {'countvectorizer__max_df': 0.85, 'countvectorizer__max_features': None, 'countvectorizer__min_df': 2, 'countvectorizer__ngram_range': (1, 2), 'multinomialnb__alpha': 2.0}
Best f1 :  0.7847455928601439


나이브 베이즈는 GridSearch를 해도 성능 향상이 보이지는 않음..
기본 모델이 가장 성능이 좋음

## XGBoost 최적화

In [ ]:
from xgboost import XGBClassifier
import xgboost as xgb
from scipy import sparse
import numpy as np
from xgboost.callback import EarlyStopping

X_train_csr = sparse.csr_matrix(X_tr).astype(np.float32)
X_val_csr   = sparse.csr_matrix(X_v).astype(np.float32)
## 여기는 object가 들어가면 안 되고 벡터화가 끝난
## 데이터가 들어가야 함!!

dtrain = xgb.DMatrix(X_train_csr, label=y_train)
dval   = xgb.DMatrix(X_val_csr,   label=y_val)

pos = (y_train == 1).sum()
neg = (y_train == 0).sum()
spw = neg / max(pos, 1)

params = {
    "objective": "binary:logistic",
    "eval_metric": "logloss",
    "eta": 0.1,                 # learning_rate
    "max_depth": 6,
    "min_child_weight": 1.0,
    "gamma": 0.0,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "reg_lambda": 1.0,
    "reg_alpha": 0.0,
    "scale_pos_weight": spw,
    "tree_method": "hist",
    "seed": 42
}

watchlist = [(dtrain, "train"), (dval, "val")]
bst = xgb.train(
    params=params,
    dtrain=dtrain,
    num_boost_round=2000,       # 충분히 크게 두고 조기종료로 컷
    evals=watchlist,
    early_stopping_rounds=50,
    verbose_eval=False
)

val_proba = bst.predict(dval, iteration_range=(0, bst.best_iteration + 1))

best_t, best_f1 = 0.5, -1.0
for t in np.linspace(0.2, 0.8, 61):
    pred_t = (val_proba >= t).astype(int)
    f1_t = f1_score(y_val, pred_t, average="macro")
    if f1_t > best_f1:
        best_f1, best_t = f1_t, t

print(f"Best threshold: {best_t:.3f} | F1(val): {best_f1:.4f}")
pred_final = (val_proba >= best_t).astype(int)
print(classification_report(y_val, pred_final))

Best threshold: 0.590 | F1(val): 0.8000
              precision    recall  f1-score   support

           0       0.79      0.90      0.84       869
           1       0.84      0.69      0.76       654

    accuracy                           0.81      1523
   macro avg       0.82      0.79      0.80      1523
weighted avg       0.81      0.81      0.81      1523



### 하이퍼 파라미터 튜닝 다른 방법 _ randomized search

In [ ]:
from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingRandomSearchCV, StratifiedKFold
from xgboost import XGBClassifier


xgb = XGBClassifier(
    objective='binary:logistic', tree_method='hist',
    eval_metric='logloss', n_jobs=-1, random_state=42
)

param_dist = {
    "learning_rate": np.logspace(-2.3, -0.7, 20),   # ~ [0.005, 0.2]
    "max_depth": np.arange(3, 10),
    "min_child_weight": np.logspace(-1, 1.3, 15),   # [0.1, 20]
    "gamma": np.linspace(0.0, 2.0, 11),
    "subsample": np.linspace(0.5, 0.9, 9),
    "colsample_bytree": np.linspace(0.5, 0.9, 9),
    "reg_alpha": np.logspace(-3, 0, 10),            # L1
    "reg_lambda": np.logspace(-2, 1, 12),           # L2
    "scale_pos_weight": [1.0, (y_train==0).sum()/max((y_train==1).sum(),1)]
}

cv = StratifiedKFold(5, shuffle=True, random_state=42)

halving = HalvingRandomSearchCV(
    estimator=xgb,
    param_distributions=param_dist,
    factor=3,                # 라운드마다 후보 1/3로 줄임
    resource='n_estimators', # 리소스 = 트리 개수
    max_resources=1200,      # 최종 라운드 트리 수
    min_resources=100,       # 초기 라운드 트리 수
    scoring='f1_macro',
    cv=cv,
    n_candidates=40,
    random_state=42,
    n_jobs=-1,
    verbose=1
)
halving.fit(X_train_csr, y_train)

n_iterations: 3
n_required_iterations: 4
n_possible_iterations: 3
min_resources_: 100
max_resources_: 1200
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 40
n_resources: 100
Fitting 5 folds for each of 40 candidates, totalling 200 fits
----------
iter: 1
n_candidates: 14
n_resources: 300
Fitting 5 folds for each of 14 candidates, totalling 70 fits
----------
iter: 2
n_candidates: 5
n_resources: 900
Fitting 5 folds for each of 5 candidates, totalling 25 fits


HalvingRandomSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
                      estimator=XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              device=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric='logloss',
                                              feature_types=None,
                                              feature_weights=None, gamma=N...
       0.04641589, 0.1       , 0.21544347, 0.46415888, 1.        ]),
                                           'reg_lambda': array([ 0.01      ,  0.01873817,  0.03511192,  0.06579332,  0.12328467,
        0.23101297,  0.43287613,  0.81113083,  1.51991108,  2.84803587,
        5.33669923, 10.        ]),
                                           'scale_pos_weight': [1.0,
                                                                np.float64(1.327092090179595)],
                                           'subsample': array([0.5 , 0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 ])},
                      random_state=42, resource='n_estimators',
                      scoring='f1_macro', verbose=1)

In [ ]:
from sklearn.metrics import f1_score, classification_report

y_val_pred = halving.best_estimator_.predict(X_val_csr)

f1_val = f1_score(y_val, y_val_pred, average='macro')
print(f"🔹 HalvingRandomSearchCV XGB F1 (val): {f1_val:.4f}")

print(classification_report(y_val, y_val_pred))

🔹 HalvingRandomSearchCV XGB F1 (val): 0.7959
              precision    recall  f1-score   support

           0       0.82      0.83      0.83       869
           1       0.77      0.76      0.77       654

    accuracy                           0.80      1523
   macro avg       0.80      0.80      0.80      1523
weighted avg       0.80      0.80      0.80      1523



## 하이퍼 파라미터 튜닝 다른 방법 _ Optuna

In [ ]:
pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 10.8 MB/s eta 0:00:00


In [ ]:
import optuna, numpy as np, xgboost as xgb
from sklearn.metrics import f1_score
from xgboost.callback import EarlyStopping

dtrain = xgb.DMatrix(X_train_csr, label=y_train)
dval   = xgb.DMatrix(X_val_csr,   label=y_val)
spw = (y_train==0).sum()/max((y_train==1).sum(),1)

def objective(trial):
    params = {
        "objective": "binary:logistic",
        "eval_metric": "logloss",
        "tree_method": "hist",
        "eta": trial.suggest_float("eta", 0.01, 0.2, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "min_child_weight": trial.suggest_float("min_child_weight", 0.1, 20, log=True),
        "gamma": trial.suggest_float("gamma", 0.0, 2.0),
        "subsample": trial.suggest_float("subsample", 0.5, 0.9),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 0.9),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-3, 1.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-2, 10.0, log=True),
        "scale_pos_weight": trial.suggest_categorical("scale_pos_weight", [1.0, float(spw)])
    }

    bst = xgb.train(
        params, dtrain,
        num_boost_round=2000,
        evals=[(dval, "val")],
        callbacks=[EarlyStopping(rounds=50, save_best=True)],
        verbose_eval=False
    )
    proba = bst.predict(dval, iteration_range=(0, bst.best_iteration+1))
    ts = np.linspace(0.2, 0.8, 19)
    f1s = [f1_score(y_val, (proba>=t).astype(int), average='macro') for t in ts]
    best = float(np.max(f1s))
    return best

study = optuna.create_study(direction='maximize', study_name='xgb_f1')
study.optimize(objective, n_trials=60, show_progress_bar=True)

print("Best params:", study.best_trial.params)
print("Best F1:", study.best_value)

[I 2025-11-11 04:32:42,826] A new study created in memory with name: xgb_f1


  0%|          | 0/60 [00:00<?, ?it/s]

[I 2025-11-11 04:32:58,497] Trial 0 finished with value: 0.7946044501113929 and parameters: {'eta': 0.09470090301484403, 'max_depth': 3, 'min_child_weight': 0.20615284116233315, 'gamma': 0.5094380075846638, 'subsample': 0.8563873516753038, 'colsample_bytree': 0.6129347302240452, 'reg_alpha': 0.6238821581037635, 'reg_lambda': 6.131659166656368, 'scale_pos_weight': 1.0}. Best is trial 0 with value: 0.7946044501113929.
[I 2025-11-11 04:32:59,162] Trial 1 finished with value: 0.6703317453474962 and parameters: {'eta': 0.10235732809005879, 'max_depth': 4, 'min_child_weight': 18.37502081149067, 'gamma': 1.222509554985512, 'subsample': 0.8632657170017, 'colsample_bytree': 0.5073655364737807, 'reg_alpha': 0.012435924981873826, 'reg_lambda': 0.02334355220997676, 'scale_pos_weight': 1.327092090179595}. Best is trial 0 with value: 0.7946044501113929.
[I 2025-11-11 04:33:04,967] Trial 2 finished with value: 0.585126265581382 and parameters: {'eta': 0.011987711648825655, 'max_depth': 8, 'min_child_